# Module 02 - 01 query multiple files using Spark

Set up variable for later reference. Make sure the name of your datalake is correct

In [2]:
datalake = 'datalake3mstaeetovkk4'

Just read in some data using CSV format 

- Notice we do not have header information in the file
- Notice we have specified to load in ALL csv files from this folder!

In [14]:
%%pyspark
df = spark.read.load('abfss://landing@' + datalake + '.dfs.core.windows.net/Allfiles/01/data/*.csv', format='csv', header=False)
display(df.limit(10))

Notice that the file does not contain any field names in the header. Let's print the schema

In [9]:
df.printSchema()

Let's define our own schema on read

In [15]:
from pyspark.sql import types

customSchema = types.StructType([
    types.StructField("SalesOrderNumber", types.StringType(), True),
    types.StructField("SalesTerritoryKey", types.IntegerType(), True),
    types.StructField("OrderDate", types.DateType(), True),
    types.StructField("Customer", types.StringType(), True),
    types.StructField("Email", types.StringType(), True),
    types.StructField("Adress", types.StringType(), True),
    types.StructField("Quantity", types.IntegerType(), True),
    types.StructField("UnitPrice", types.DoubleType(), True),
    types.StructField("ShippingCost", types.DoubleType(), True),
])

df = spark.read \
    .csv('abfss://landing@' + datalake + '.dfs.core.windows.net/Allfiles/01/data/*.csv', schema=customSchema)

display(df.limit(10))

Now, let's filter this data using Spark SQL. First we register this table as a temporary view

In [16]:
df.createOrReplaceTempView("sales")

And now we can apply the **%%sql** magic

In [17]:
%%sql

SELECT * 
FROM SALES 
WHERE Customer LIKE 'E%'

We can also apply grouping and mathematical operations like SUM. 

In [22]:
%%sql

SELECT 
    OrderDate,
    SUM( (Quantity * UnitPrice) + ShippingCost ) AS TotalSales
FROM Sales
GROUP BY OrderDate
ORDER BY OrderDate
LIMIT 50

Create a new temporary view by using SQL:

In [23]:
%%sql

CREATE OR REPLACE TEMPORARY VIEW TotalSalesByDate
AS

SELECT 
    OrderDate,
    SUM( (Quantity * UnitPrice) + ShippingCost ) AS TotalSales
FROM Sales
GROUP BY OrderDate
ORDER BY OrderDate

... and then use the temporary view

In [27]:
%%sql

SELECT * FROM TotalSalesByDate LIMIT 10